In [ ]:
# prompt: connect drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/drive/MyDrive/Final Data.csv'  # Update with your file path
data = pd.read_csv(file_path)

# List of column names to convert into a corpus
columns_to_combine = [
    'S.No', 'District', 'Division', 'Police Station', 'Hotel', 'Longitude',
    'Latitude', 'Manager Name', 'Address', 'Phone#',
    'Additional_Number_of_Scoring', 'Review_Date', 'Average_Score',
    'Reviewer_Nationality', 'Negative_Review',
    'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
    'Positive_Review', 'Review_Total_Positive_Word_Counts',
    'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
    'Tags', 'days_since_review'
]

# Ensure all columns exist in the dataset
for col in columns_to_combine:
    if col not in data.columns:
        print(f"Warning: Column '{col}' not found in the dataset. Please check your file.")
        exit()

# Combine the specified columns into a single corpus column
data['corpus'] = data[columns_to_combine].astype(str).agg(' '.join, axis=1)

# Save the new dataset with the corpus
output_file = '/content/dataset_with_corpus.csv'
data.to_csv(output_file, index=False)

print(f"Dataset with corpus saved to: {output_file}")


Dataset with corpus saved to: /content/dataset_with_corpus.csv


In [ ]:
# prompt: data embding

!pip install sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer

# Load the dataset
file_path = '/content/dataset_with_corpus.csv'  # Update with your file path
data = pd.read_csv(file_path)

# Select the 'corpus' column for embedding
corpus_column = 'corpus'

# Check if the corpus column exists
if corpus_column not in data.columns:
    print(f"Error: Column '{corpus_column}' not found in the dataset. Please check your file.")
    exit()


# Initialize the Sentence Transformer model
model = SentenceTransformer('all-mpnet-base-v2')  # You can choose other models

# Generate embeddings for the corpus
corpus_embeddings = model.encode(data[corpus_column].tolist(), show_progress_bar=True)


# Create a DataFrame to store the embeddings
embeddings_df = pd.DataFrame(corpus_embeddings)

# Concatenate the embeddings with the original DataFrame
data = pd.concat([data, embeddings_df], axis=1)

# Save the data with embeddings
output_file = '/content/dataset_with_embeddings.faiss'
data.to_csv(output_file, index=False)

print(f"Dataset with embeddings saved to: {output_file}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Dataset with embeddings saved to: /content/dataset_with_embeddings.faiss


In [ ]:
# prompt: next code for LLM model artecture

!pip install faiss-cpu

import faiss

# Load the dataset with embeddings
file_path = '/content/dataset_with_embeddings.faiss'  # Update with your file path
data = pd.read_csv(file_path)

# Separate embeddings from other data
embeddings = data.iloc[:, -768:].values.astype('float32')  # Assuming 768 dimensions
data_without_embeddings = data.iloc[:, :-768]

# Build the Faiss index
d = embeddings.shape[1]  # Dimension of the vectors
index = faiss.IndexFlatL2(d)  # Use IndexFlatL2 for exact search
index.add(embeddings)

# Example query (replace with actual query)
query = "hotel in Greece with good reviews"  # Example query

# Encode the query using the same Sentence Transformer model
query_embedding = model.encode(query)
query_embedding = query_embedding.reshape(1, -1).astype('float32')

# Perform the search
k = 5  # Number of nearest neighbors to retrieve
D, I = index.search(query_embedding, k)

# Print the results
print("Distances:", D)
print("Indices:", I)

# Retrieve the top-k results from the original dataset
results = data_without_embeddings.iloc[I[0]]

# Display the results (you can customize the output)
results

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 35.6 MB/s eta 0:00:00
Distances: [[0.8506632  0.8569641  0.86728215 0.89257586 0.89880717]]
Indices: [[157  26 208  21 903]]


S.No District Division Sub-Division Police Station  \
157   158  Ghanche  Ghanche         City      PS Khaplu   
26     27   Astore   Astore      Shonter      PS Guddai   
208   209   Ghizer   Ghizer   SDPO GUPIS     PS Phandar   
21     22   Astore   Astore      Shonter       PS Rattu   
903   904    Nagar    NAGAR   SDPO NAGAR     PS Minapin   

                             Hotel  Longitude  Latitude     Manager Name  \
157  High Cliff Guest House Khaplu       74.5      35.1  Ghulam Muhammad   
26                Karakuram Hotel        74.5      35.5    Nasir Hussain   
208             Dream Island Hotel       73.5      36.5      Mir shahbaz   
21          Rupal Resort Tarishing       74.5      35.5         Jahanzeb   
903            AlQaim Guest House        74.0      36.3    Abdul Mutalib   

                     Address  ...  \
157        VIP Colony Khaplu  ...   
26      Chillum  deosai Road  ...   
208  Phander District Ghizer  ...   
21          Tarishing Astore  ...   
903     Masoot Gulmit  Nagar  ...   

                                       Negative_Review  \
157   The first night we got there we where put in ...   
26    Nothing at all to do with the Hotel of course...   
208   The only negative was the noise there were bu...   
21    Our room was an overrated disaster room 231 d...   
903                                        No Negative   

     Review_Total_Negative_Word_Counts Total_Number_of_Reviews  \
157                                145                    1403   
26                                  51                    1403   
208                                 65                    1403   
21                                  92                    1403   
903                                  0                    1831   

                                       Positive_Review  \
157   The only good thing about this hotel is the f...   
26    The Hotel itself is in a lovely location a 5m...   
208   The staff were extremely polite and helpful i...   
21    The breakfast was the only positive element o...   
903   Location cleanliness friendly staff A nice qu...   

    Review_Total_Positive_Word_Counts  \
157                                94   
26                                134   
208                                27   
21                                 75   
903                                10   

    Total_Number_of_Reviews_Reviewer_Has_Given  Reviewer_Score  \
157                                          1             5.0   
26                                           2             9.6   
208                                          1             7.9   
21                                          12             3.8   
903                                          5             8.8   

                                                  Tags days_since_review  \
157  [' Leisure trip ', ' Couple ', ' Duplex Double...           509 day   
26   [' Leisure trip ', ' Group ', ' Duplex Double ...           70 days   
208  [' Leisure trip ', ' Couple ', ' Duplex Double...           647 day   
21   [' Couple ', ' Duplex Double Room ', ' Stayed ...           52 days   
903  [' Couple ', ' Classic Double Room ', ' Stayed...           555 day   

                                                corpus  
157  158 Ghanche Ghanche PS Khaplu High Cliff Guest...  
26   27 Astore Astore PS Guddai Karakuram Hotel  74...  
208  209 Ghizer Ghizer PS Phandar Dream Island Hote...  
21   22 Astore Astore PS Rattu Rupal Resort Tarishi...  
903  904 Nagar NAGAR PS Minapin AlQaim Guest House ...  

[5 rows x 25 columns]

In [ ]:
tokenized = tokenizer(text)

NameError: name 'tokenizer' is not defined

In [ ]:
pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25


In [ ]:
GROQ_API_KEY="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"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [ ]:
import langchain

In [ ]:
chat = ChatGroq(temperature=0, groq_api_key="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", model_name="mixtral-8x7b-32768")


In [ ]:
system = "Name of hotels in Gilgit Baltistan"
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat

ans = chain.invoke({"text": "cost less hotel in Gilgit."})

In [ ]:
ans.content

"Here are some hotel names in Gilgit, Pakistan that are known for being affordable:\n\n1. Gilgit Serena Hotel\n2. Hotel Royal Palace\n3. PTDC Motel Gilgit\n4. Summit Hotel\n5. Riveria Hotel\n6. Hotel Hill View\n7. Pine Park Hotel\n8. Mascot Hotel\n9. Tourist Inn Hotel\n10. New Golden Peak Hotel\n\nPlease note that prices can vary depending on the time of year and availability, so it's always a good idea to check the hotel's website or contact them directly for the most up-to-date pricing information."

In [ ]:
ans = list() # or ans = []
# Or if 'ans' is already defined and you want to add a list of hotels to it, you might do:
# ans.extend(list_of_hotels)
# Assuming list_of_hotels is defined somewhere.

In [ ]:
def generate_query_embedding(Query):
  """
  Generates an embedding for a given query using a pre-trained language model.

  Args:
    Query: The input query string.

  Returns:
    A NumPy array representing the embedding of the query.
  """

  # Tokenize the query
  inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)

  # Generate embedding using the language model
  with torch.no_grad():
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling

  return embedding

def recommend_hotels(Query, top_k=5):
  """
  Recommends hotels based on a given query.

  Args:
    Query: The input query string.
    top_k: The number of top recommendations to return.

  Returns:
    A list of dictionaries, where each dictionary contains:
      - "Hotel Details": Information about the recommended hotel.
      - "Score": The similarity score between the query and the hotel.
  """

  query_embedding = generate_query_embedding(Query)

  # Find the nearest neighbors in the Faiss index
  distances, indices = faiss_index.search(query_embedding, k=top_k)

  recommendations = []
  for i in range(top_k):
    hotel_details = hotel_data[indices[i]]  # Assuming hotel_data is your list of hotels
    score = distances[i]
    recommendations.append({"Hotel Details": hotel_details, "Score": score})

  return recommendations

In [ ]:

# Step 3: Function for Querying Recommendations
def generate_query_embedding(Query):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()  # Mean pooling
    return embedding

def recommend_hotels(Query, top_k=5):
    query_embedding = generate_query_embedding(Query)
    results = faiss_index.similarity_search_with_score(query_embedding, k=top_k)
    recommendations = [
        {"Hotel Details": result[0], "Score": result[1]} for result in results
    ]
    return recommendations

# Step 4: Build a Chatbot Interface
def chatbot():
    print("Welcome to the Hotel Recommendation Chatbot!")
    print("You can ask me for Hotel Recommendations. Type 'exit' to quit.")

    while True:
        user_input = input("\nYou: ")

        if user_input.lower() in ['exit', 'quit']:
            print("Chatbot: Thank you for using the Hotel Recommendation Chatbot. Goodbye!")
            break

        print("Chatbot: Let me find some recommended hotels for you...")

        # Get recommendations
        try:
            recommendations = recommend_hotels(user_input)
            print("\nHere are the top recommended hotels for you:")
            for idx, rec in enumerate(recommendations, 1):
                print(f"{idx}. {rec['Hotel Details']} (Score: {rec['Score']:.4f})")
        except Exception as e:
            print(f"Chatbot: Sorry, something went wrong. {str(e)}")

# Step 5: Run the Chatbot
if __name__ == "__main__":
    chatbot()

Welcome to the Hotel Recommendation Chatbot!
You can ask me for Hotel Recommendations. Type 'exit' to quit.
Chatbot: Let me find some recommended hotels for you...
Chatbot: Sorry, something went wrong. name 'tokenizer' is not defined

You: exit
Chatbot: Thank you for using the Hotel Recommendation Chatbot. Goodbye!


In [ ]:
!pip install gradio
import gradio as gr
# A simple function for the chatbot
def chatbot_response(user_input):
    # Replace this with your chatbot logic
    if "hello" in user_input.lower():
        return "Hi there! How can I help you?"
    elif "hotel" in user_input.lower():
        return "chat"
    else:
        return " yes i'll help you for recommended hotels , I did understand that.?"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
# Create Gradio Interface
# Move the call_chat definition here (before gr.Interface)
def call_chat(Question):
    system = "You are a hotels recommender for Gilgit baltistan. Answer for only what is asked. Do not suppose on your own."
    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", Question)])

    chain = prompt | chat
    Ans = chain.invoke({"text": "cost less hotel in Gilgit Baltistan."})
    return ans.content

interface = gr.Interface(
    fn=call_chat,  # Function to call
    inputs="text",        # Input type
    outputs="text",       # Output type
    title="Hotel Recommendation Chatbot",  # App Title
    description="Ask me About Hotels and Recommendations!",  # App Description
)

In [ ]:
# Launch the Gradio app
interface.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ffdf724910003830a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
